# 1. Подготовка

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
import numpy as np

In [ ]:
a=['I only can kill a mosquito','quarantine made me feel unnecessary and stupid :(','I will be happy if you make me cry','i don’t want to be like you',"I can't watch you try to attract attention «"]

comments=pd.DataFrame(data=a, columns=['text'])

comments

In [2]:
data =  pd.read_csv('/datasets/toxic_comments.csv')
data


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


Данные загружены. Язык английский.

In [ ]:
#nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

Установим список стоп-слов на английском.

In [ ]:
def clear_text(text):
    raw_text=re.sub(r"[^a-zA-Z' ]", ' ', text)
    return " ".join(raw_text.split())

data['lemm'] = data.text.apply(clear_text)
data

In [ ]:
comments['lemm'] = comments.text.apply(clear_text)
comments

Оставим в комментариях только буквы, единичные пробелы и апостроф '.

In [ ]:
def lemmatize_text(text):
    return " ".join(lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text))

data['lemm'] = data.lemm.apply(lemmatize_text)
data

In [12]:
data['text'].loc[5]

'"\n\nCongratulations from me as well, use the tools well. \xa0· talk "'

In [ ]:
comments['lemm'] = comments.lemm.apply(lemmatize_text)
comments

Лемматизируем очищенный столбец.

In [ ]:
data_valid_train, data_test = train_test_split(data, test_size=0.1, random_state=12345)
data_train, data_valid = train_test_split(data_valid_train,test_size=0.1, random_state=12345)


corpus_train=data_train['lemm'].values.astype('U')
corpus_valid=data_valid['lemm'].values.astype('U')
corpus_test=data_test['lemm'].values.astype('U')




In [ ]:
corpus_comm=comments['lemm'].values.astype('U')

Разобьем данные на обучающую, валидационную и тестовую выборки. Создадим 3 корпуса соответственно.

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)


tf_idf_train = count_tf_idf.fit_transform(corpus_train)#train features



tf_idf_valid = count_tf_idf.transform(corpus_valid)#valid features



tf_idf_test = count_tf_idf.transform(corpus_test)#test features


In [ ]:
tf_idf_comm= count_tf_idf.transform(corpus_comm)#test features


Применим Tf-id к корпусам, чтобы получить признаки. Целевой признак - столбец toxic, его не меняли.

In [ ]:
features_train = tf_idf_train
target_train = data_train['toxic']

features_valid = tf_idf_valid
target_valid = data_valid['toxic']

features_test = tf_idf_test
target_test = data_test['toxic']

In [ ]:
features_comm = tf_idf_comm

Признаки и целевой признак обозначены.

# 2. Обучение

In [ ]:
print('Логистическая регрессия')

for i in np.arange(1, 1.5, 0.1):
    model = LogisticRegression(random_state=12345,C=i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    print('C=', i, ':',f1_score(target_valid, predicted_valid))



Подберем гиперпараметры логистической регрессии.

In [ ]:
print('Дерево решений')

for i in range(1, 12):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    print("max_depth =", i, ":", f1_score(target_valid, predicted_valid))



Подберем гиперпараметры дерева решений.

Наилучший результат показала регрессия с С=1,4, даную модель проверим на тестовой выборке.

# 3. Выводы

In [ ]:
model = LogisticRegression(random_state=12345,C=1.4, class_weight='balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)

print(f1_score(target_test, predicted_test))

In [ ]:
model = LogisticRegression(random_state=12345,C=1.4, class_weight='balanced')
model.fit(features_train, target_train)
predicted_comm = model.predict(features_comm)

print(predicted_comm)

comments['toxic'] = predicted_comm

print()
print(comments)

Значение f1 больше о,75. 
## Вывод: итоговая модель - логистическая регрессия с параметрами: C=1.4, class_weight='balanced'.